In [1]:
# Test de Turing Interactivo - Evaluación perceptiva de textos

import pandas as pd
import random
from IPython.display import display, Markdown

# Cargar dataset completo con textos y clases (ajustar nombre si es distinto)
df = pd.read_csv("dataset_final_completo.csv")
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # eliminar columnas vacías si las hubiera

# Simulación de predicciones (si ya tienes predicciones reales, puedes cargarlas)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Tokenizador y modelo de BERT
model = DistilBertModel.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Función para extraer embeddings BERT

def get_bert_embeddings(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        embeddings.append(cls_embedding)
    return embeddings

# Separar datos para simulación
X = df["texto"]
y = df["clase"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Obtener embeddings
X_train_emb = get_bert_embeddings(X_train.tolist())
X_test_emb = get_bert_embeddings(X_test.tolist())

# Entrenar modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_emb, y_train)
y_pred = clf.predict(X_test_emb)

# Preparar DataFrame de prueba
resultados = pd.DataFrame({
    "texto": X_test.reset_index(drop=True),
    "real": y_test.reset_index(drop=True),
    "pred": y_pred
})

# Interfaz interactiva para evaluación manual
num_muestras = 5  # puedes cambiar este valor
df_eval = resultados.sample(n=num_muestras, random_state=1).reset_index(drop=True)

for i in range(num_muestras):
    display(Markdown(f"### \U0001f9e0 Texto {i+1}\n{df_eval.loc[i, 'texto']}"))
    respuesta = input("\u00bfCrees que este texto es HUMANO o IA? (escribe 'humano' o 'ia'): ").strip().lower()
    real = df_eval.loc[i, "real"]
    pred = df_eval.loc[i, "pred"]

    print(f"\n\u2705 Tu respuesta: {respuesta}")
    print(f"\U0001f9ea Modelo predijo: {pred}")
    print(f"\U0001f3af Realidad: {real}")

    if respuesta == real:
        print("\u2705 Coincide con la clase real.")
    else:
        print("\u274c No coincide con la clase real.")

    print("-" * 100)

c:\Users\ddiaz\OneDrive\Documentos\UNIVERSIDAD EUROPEA\ENTREGABLES\TFM\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 🧠 Texto 1
Nyria alzó el grimorio sobre la piedra rúnica, mientras el viento enloquecía a su alrededor y las brasas del bosque encantado se elevaban en espirales anaranjadas. El fuego no quemaba, sino que susurraba nombres olvidados, secretos que dormían bajo la montaña de ceniza. Había creído que su linaje de sangre estaba extinguido, pero el códice la había elegido. Y una vez leído en voz alta, no habría marcha atrás: el Fuego Eterno regresaría.


✅ Tu respuesta: ia
🧪 Modelo predijo: humano
🎯 Realidad: ia
✅ Coincide con la clase real.
----------------------------------------------------------------------------------------------------


### 🧠 Texto 2
Las civilizaciones antiguas han dejado un impacto profundo en la sociedad moderna, desde las leyes hasta el arte. Mesopotamia nos legó la escritura, Egipto la arquitectura monumental, Grecia la filosofía y Roma el derecho. Aunque algunas de estas culturas han desaparecido, sus conocimientos siguen presentes en nuestro día a día, moldeando nuestra forma de ver el mundo. Sin embargo, la pregunta persiste: ¿hasta qué punto hemos logrado evolucionar sin olvidar nuestras raíces? La historia no solo debe ser estudiada, sino comprendida para evitar repetir los errores del pasado y fortalecer los logros que nos definen como humanidad.


✅ Tu respuesta: humano
🧪 Modelo predijo: ia
🎯 Realidad: ia
❌ No coincide con la clase real.
----------------------------------------------------------------------------------------------------


...


✅ Tu respuesta: humano
🧪 Modelo predijo: humano
🎯 Realidad: humano
✅ Coincide con la clase real.
----------------------------------------------------------------------------------------------------


### 🧠 Texto 4
El presidente de Colombia, Gustavo Petro, ha exigido explicaciones al Gobierno de España por un incidente ocurrido el 17 de mayo en un restaurante de Valencia, donde una familia migrante colombiana fue presuntamente agredida de forma excesiva por agentes de la Policía Nacional.


✅ Tu respuesta: humano
🧪 Modelo predijo: humano
🎯 Realidad: humano
✅ Coincide con la clase real.
----------------------------------------------------------------------------------------------------


### 🧠 Texto 5
La nostalgia no es solo un sentimiento, es también una herramienta política. En tiempos de incertidumbre, recurrimos al pasado como refugio: idealizamos lo que fue, simplificamos lo que recordamos. Desde las series que reviven los años ochenta hasta los discursos que prometen “volver a ser grandes”, la cultura contemporánea ha convertido el anhelo del ayer en mercancía. Este ensayo analiza cómo esa mirada nostálgica moldea nuestra forma de votar, consumir y pensar el futuro.


✅ Tu respuesta: ia
🧪 Modelo predijo: ia
🎯 Realidad: ia
✅ Coincide con la clase real.
----------------------------------------------------------------------------------------------------
